In [0]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import *
from collections import Counter
import sys
import jsonlines

In [0]:
"""
Add API keys to access Twitter API
"""
consumer_key=""
consumer_secret=""
access_token=""
access_token_secret=""

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth)

target = '@midasIIITD'
print(target)

item = auth_api.get_user(target)
tweets = item.statuses_count

In [0]:
"""
Get tweets (not pages) from Cursor objects and push entire string onto a JSON string.
Then write all JSON strings to a JSONLines file, as shown
"""
tweet_count = 0
tweets = []
end_date = datetime.utcnow() - timedelta(days=1000)
for status in Cursor(auth_api.user_timeline, screen_name='@midasIIITD', tweet_mode="extended", id=target).items():
	tweet_count+=1
	tweets.append(status._json)
	"""
	Add code to get relevant details
	"""
	if status.created_at < end_date:
		break
with jsonlines.open('temp.jsonl', mode='w') as writer:
writer.write_all(tweets)

In [0]:
import pandas as pd
import jsonlines
import json
"""
Pending:
Done:
	1. Add image count
		@Note: Retweet image are not to be included!
		Look in entities->media only.
"""
tweets = []

In [0]:
"""
Read jsonl file and collate all tweets. 
Create .csv file from relevant headers.
"""
with jsonlines.open('temp.jsonl', mode='r') as reader:
	for obj in reader.iter(type=dict, skip_invalid=True):
		tweets.append(obj)

tweets_data = pd.DataFrame(tweets)

actual = pd.DataFrame()
actual['text'] = tweets_data['full_text']
actual['date'] = tweets_data['created_at']
actual['favorite_count'] = tweets_data['favorite_count']
actual['retweets'] = tweets_data['retweet_count']
#actual['images'] = len(tweets_data['extended_entities']['media'])
actual['images'] = 0
for i in range(len(tweets)):
	try:
		actual['images'][i] = len(tweets_data['extended_entities'][i]['media'])
	except Exception as e:
		actual['images'][i] = 0

print(actual)

actual.to_csv('midas_twitter_feed_stats.csv')